In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time

# WebDriver 초기화 및 암시적 대기 시간 설정
driver = webdriver.Chrome()
driver.implicitly_wait(10)

# 대상 웹 페이지 접속
url = 'https://www.consumer.go.kr/user/ftc/consumer/recallInfo/629/selectRecallInfoInternalList.do?searchCondition1=0301'
driver.get(url)

image_xpaths = [
    '//*[@id="goodsList"]/ul/li[1]/div[1]/a/p/img',
    '//*[@id="goodsList"]/ul/li[2]/div[1]/a/p/img',
    '//*[@id="goodsList"]/ul/li[3]/div[1]/a/p/img',
    '//*[@id="goodsList"]/ul/li[4]/div[1]/a/p/img',
    '//*[@id="goodsList"]/ul/li[5]/div[1]/a/p/img',
    '//*[@id="goodsList"]/ul/li[6]/div[1]/a/p/img',
    '//*[@id="goodsList"]/ul/li[7]/div[1]/a/p/img',
    '//*[@id="goodsList"]/ul/li[8]/div[1]/a/p/img',
    '//*[@id="goodsList"]/ul/li[9]/div[1]/a/p/img',
    '//*[@id="goodsList"]/ul/li[10]/div[1]/a/p/img'
]

data = []
# 각 이미지에 대해 순회하며 클릭하고 데이터 수집
for xpath in image_xpaths:
    try:
        # 이미지 요소 찾기
        image = driver.find_element(By.XPATH, xpath)

        # 이미지가 클릭 가능하도록 스크롤
        driver.execute_script("arguments[0].scrollIntoView();", image)
        time.sleep(1)  # 스크롤 후 로딩 시간을 고려하여 잠시 대기

        # 이미지 클릭
        ActionChains(driver).move_to_element(image).click().perform()
        time.sleep(2)  # 페이지 로딩 시간을 고려하여 잠시 대기

        # 리콜 정보가 담긴 테이블 추출
        table = driver.find_element(By.XPATH, '//*[@id="m_page_top"]/div[2]/div[2]')
        soup = BeautifulSoup(table.get_attribute("innerHTML"), 'html.parser')
        recall_table = soup.find("div", class_="goodsViewUi")

        if recall_table:
            rows = recall_table.find_all('dl')
            for row in rows:
                key = row.find('dt').get_text(strip=True)
                value = '\n'.join(p.get_text(strip=True) for p in row.find('dd').find_all('p')) if row.find('dd').find_all('p') else row.find('dd').get_text(strip=True)
                data.append({key: value})
        else:
            print("리콜 정보를 담은 테이블을 찾을 수 없습니다.")

        # 이전 페이지로 돌아가기
        driver.back()
        time.sleep(2)  # 페이지 로딩 시간을 고려하여 잠시 대기
   
    except Exception as e:
        print(f'오류 발생: {e}')
        continue

print(data)

# 작업 완료 후 드라이버 종료
driver.quit()

[{'리콜구분': '자발적리콜'}, {'리콜공표기간': '2025.04.01 ~ 2026.10.01'}, {'출처': '국토교통부'}, {'결함의 내용': '1) 자동차의 외부에서 승강구를 열기 위한 장치는 지상으로부터 1,000mm부터 1,500mm까지 높이이고, 승강구로부터 500mm 이내에 있어야 하나 기준에 미달하여 자동차 및 자동차부품의 성능과 기준에 관한 규칙 제29조를 위반함이 확인됨에 따른 리콜\n\n2) 내부 조작 장치는 승강구로부터 300mm 이내 설치되어 있어야 하나 기준에 초과하여 자동차 및 자동차부품의 성능과 기준에 관한 규칙 제29조를 위반함이 확인됨에 따른 리콜\n\n<대상차량별 생산년월일>\n1) 승강구 조작 장치(외부): 2021.12.01~2022.10.31\n2) 승강구 조작 장치(내부): 2021.12.01~2023.11.08'}, {'리콜구분': '자발적리콜'}, {'리콜공표기간': '2025.04.01 ~ 2026.10.01'}, {'출처': '국토교통부'}, {'결함의 내용': '먼지와 이물질로 인해 2차 후드걸쇠제어 장치의 레버 피벗 조인트에 이물질이 쌓일 경우 마찰 또는 장기적으로 부식 가능성을 유발할 수 있고 이로 인해 운전자들이 차량 점검을 위해 2차 후드걸쇠장치를 해제한 후 후드를 닫아도 2차 후드걸쇠장치 레버가 열린 위치에서 고착이 될 수 있음. 2차 후드걸쇠제어 장치의 레버가 열린 위치에서 고착이 된 경우 1차 후드걸쇠장치를 해제 하거나 후드가 제대로 닫히지 않고 이동 중인 경우에 후드가 경고 없이 열려 운전자의 전방 시야를 방해할 수 있으며, 이로 인해 충돌 위험이 증가할 수 있는 가능성에 따른 리콜'}, {'리콜구분': '자발적리콜'}, {'리콜공표기간': '2025.04.01 ~ 2026.10.01'}, {'출처': '국토교통부'}, {'결함의 내용': '[사전공지] 이피션트크루즈 기능 비활성화를 안할 시 특정지역을 통과할 때 경고등과 함게 기어박스가 영향을 받아 기어가 D(드라이브)모드에서 N(중립단